In [276]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VAR
from scipy.stats import norm
from sklearn.metrics import r2_score

In [277]:
# Laste poll-of-polls data
df = pd.read_csv("../data/all_v2_data.csv", sep=",")

In [278]:
df.dropna(inplace=True)

In [279]:
# Sort values and set index
df = df.sort_values("Mnd")
df.set_index("Mnd", inplace=True)

In [280]:
df_diff = df.diff().dropna()  # Difference to make stationary

In [281]:
df_en=df[['Ap', 'Hoyre', 'Frp', 'SV', 'SP', 'KrF', 'Venstre', 'MDG', 'Rodt','Andre']]

In [312]:
df_ex=df[[
        'Ap_skandale', 'Hoyre_skandale', 'Frp_skandale', 'SV_skandale',
       'SP_skandale', 'KrF_skandale', 'Venstre_skandale', 'MDG_skandale',
       'Rodt_skandale', 'Andre_skandale', 'Ap_skandale_lag3',
       'Ap_skandale_lag6', 'Hoyre_skandale_lag6', 'Frp_skandale_lag6',
       'SV_skandale_lag6', 'SP_skandale_lag6', 'KrF_skandale_lag6',
       #'Venstre_skandale_lag6', 'MDG_skandale_lag6', 
        'Rodt_skandale_lag6',
       #'Andre_skandale_lag6', 
       # 'Ap_reg', 'Hoyre_reg', 'Frp_reg', 'SV_reg',
       #'SP_reg', 'KrF_reg', 'Venstre_reg', 'MDG_reg', 'Rodt_reg', 'Andre_reg',
       #'Ap_reg_lag3', 
       'Ap_reg_lag6',
        'Hoyre_reg_lag6', 
        'Frp_reg_lag6','SV_reg_lag6', 
        'SP_reg_lag6','KrF_reg_lag6',
        'Venstre_reg_lag6', 
        #'MDG_reg_lag6',
        #'Rodt_reg_lag6', 
        #'Andre_reg_lag6',
       'ledighet', 'ledig_trend3', 'ledig_trend6', 'styringsrente',
       'styringsrente_trend3', 'styringsrente_trend6']]

In [313]:
df_ex

,Ap_skandale,Hoyre_skandale,Frp_skandale,SV_skandale,SP_skandale,KrF_skandale,Venstre_skandale,MDG_skandale,Rodt_skandale,Andre_skandale,...,SV_reg_lag6,SP_reg_lag6,KrF_reg_lag6,Venstre_reg_lag6,ledighet,ledig_trend3,ledig_trend6,styringsrente,styringsrente_trend3,styringsrente_trend6
Mnd,,,,,,,,,,,,,,,,,,,,,
1/31/2009,0,0,0,0,0,0,0,0,0,0,...,6.0,6.0,0.0,0.0,3.2,0.033,0.033333,3.00,-0.583333,-0.458333
1/31/2010,0,0,0,0,0,0,0,0,0,0,...,6.0,6.0,0.0,0.0,3.9,0.067,0.083333,1.75,0.083333,0.083333
1/31/2011,0,0,0,0,0,0,0,0,0,0,...,6.0,6.0,0.0,0.0,3.5,-0.033,-0.066667,2.00,0.000000,0.000000
1/31/2012,0,0,0,1,0,0,0,0,0,0,...,6.0,6.0,0.0,0.0,3.6,0.000,0.016667,1.75,-0.166667,-0.083333
1/31/2013,0,0,0,0,0,0,0,0,0,0,...,6.0,6.0,0.0,0.0,3.8,0.033,0.066667,1.50,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9/30/2020,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,6.0,6.0,5.3,-0.033,0.183333,0.00,0.000000,-0.041667
9/30/2021,0,0,0,0,0,1,0,0,0,0,...,0.0,0.0,6.0,6.0,3.8,-0.233,-0.200000,0.06,0.020000,0.010000
9/30/2022,0,0,0,0,0,0,0,0,0,0,...,0.0,6.0,0.0,0.0,3.3,0.033,0.033333,1.89,0.213333,0.190000


In [314]:
model = VAR(endog=df_en, exog=df_ex)
#model = VAR(df_en)
model_fitted = model.fit(6)

print(model_fitted.summary())

C:\Users\jens.nilsen\python\WPy64-31230\python-3.12.3.amd64\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\jens.nilsen\python\WPy64-31230\python-3.12.3.amd64\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


LinAlgError: Singular matrix